In [1]:
%pip install Mastodon.py

                                              0.0/64.5 kB ? eta -:--:--
     ---------------------------------------- 64.5/64.5 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


https://github.com/mastodon/mastodon/

https://mastodonpy.readthedocs.io/en/stable/02_return_values.html#user-account-dicts


In [1]:
from mastodon import Mastodon


api_base_url = 'https://mastodon.social'

'''
Mastodon.create_app(
    'acmbrito_app',
    api_base_url = api_base_url,
    to_file = 'acmbrito_app_python_secret'
)'''


"\nMastodon.create_app(\n    'acmbrito_app',\n    api_base_url = api_base_url,\n    to_file = 'acmbrito_app_python_secret'\n)"

In [2]:
mastodon_app = open('acmbrito_app_python_secret').readlines()
client_id = mastodon_app[0]
client_secret = mastodon_app[1]
mastodon = Mastodon(client_id = client_id, client_secret = client_secret, api_base_url = api_base_url)


In [6]:
user_server = "scholar.social"
user_name = "Alex_Golub"

# profile_json = 'https://{}/api/v1/accounts/lookup?acct={}'.format(user_server, user_name)

mastodon.account_lookup("{}@{}".format(user_name, user_server))
# para conseguir os seguidores: GET /api/v1/accounts/:id/followers HTTP/1.1

{'id': 309470,
 'username': 'Alex_Golub',
 'acct': 'Alex_Golub@scholar.social',
 'display_name': 'Alex Golub',
 'locked': False,
 'bot': False,
 'discoverable': True,
 'group': False,
 'created_at': datetime.datetime(2018, 3, 28, 0, 0, tzinfo=tzutc()),
 'note': '<p>Professor of <a href="https://scholar.social/tags/Anthropology" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>Anthropology</span></a> at the University of Hawai‘i at Mānoa. <a href="https://scholar.social/tags/PNG" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>PNG</span></a> <a href="https://scholar.social/tags/PapuaNewGuinea" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>PapuaNewGuinea</span></a> <a href="https://scholar.social/tags/Mining" class="mention hashtag" rel="nofollow noopener noreferrer" target="_blank">#<span>Mining</span></a> <a href="https://scholar.social/tags/HIstoryofAnthropology" class="mention hasht